<a href="https://colab.research.google.com/github/Dannynis/mp3_superresolution/blob/main/mp3_superresolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
cd /content/drive/MyDrive/mp3_superresulution/dataset/fma

/content/drive/MyDrive/mp3_superresulution/dataset/fma


In [2]:
# ls data/mp3s/

In [3]:
# prompt: using multithreading copy all the files from data/mp3s/320mp3 and data/mp3s/128mp3  to /content

# import os
# import shutil
# import threading
# from concurrent.futures import ThreadPoolExecutor
# import tqdm
# def copy_files(source_dir, destination_dir):
#   tp = ThreadPoolExecutor(max_workers=10)
#   files = os.listdir(source_dir)
#   futures = []
#   for file in files:
#     source_path = os.path.join(source_dir, file)
#     source_dir_name = os.path.basename(source_dir)
#     destination_path = os.path.join(destination_dir, source_dir_name, file)
#     if os.path.exists(destination_path):
#       continue
#     futures.append( tp.submit(shutil.copy, source_path, destination_path))
#   for future in tqdm.tqdm_notebook(futures):
#     future.result()

# source_dirs = ["data/mp3s/320mp3", "data/mp3s/128mp3"]
# destination_dir = "/content"

# for source_dir in source_dirs:
#   os.makedirs(os.path.join(destination_dir, os.path.basename(source_dir)), exist_ok=True)
#   copy_files(source_dir,destination_dir)
# threads = []
# for source_dir in source_dirs:
#   thread = threading.Thread(target=copy_files, args=(source_dir, destination_dir))
#   threads.append(thread)
#   thread.start()

# for thread in threads:
#   thread.join()


## Training

In [4]:
pip install pydub

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/mp3_superresulution/denoiser/')

In [5]:
# prompt: generate pytorch dataset that loads pairs of audios with the same name from data/mp3s/320mp3 and data/mp3s/128mp3

import os
import torch
from torch.utils.data import Dataset
import torchaudio
import soundfile as sf
from concurrent.futures import ThreadPoolExecutor
import tqdm
from pydub import AudioSegment
import pydub
import shutil

class PairedAudioDataset(Dataset):
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.mp3s_320 = os.listdir(os.path.join(root_dir, 'data/mp3s/320mp3'))
    self.mp3s_128 = os.listdir(os.path.join(root_dir, 'data/mp3s/128mp3'))
    os.makedirs('/content/data/mp3s/320mp3/', exist_ok=True)
    os.makedirs('/content/data/mp3s/128mp3/', exist_ok=True)
    self.tp = ThreadPoolExecutor(max_workers=10)


  def save_file(self, src, dst):
    shutil.copy(src, dst)

  def __len__(self):
    return len(self.mp3s_320) * 2

  def __getitem__(self, idx):
    try:
      mp3_name = self.mp3s_320[idx // 2]
      if os.path.exists(os.path.join('/content/data/mp3s/320mp3/', mp3_name)) and os.path.exists(os.path.join('/content/data/mp3s/128mp3/', mp3_name)):
        root_dir = '/content'
      else:
        root_dir = self.root_dir
      mp3_path_320 = os.path.join(root_dir, 'data/mp3s/320mp3', mp3_name)
      mp3_path_128 = os.path.join(root_dir, 'data/mp3s/128mp3', mp3_name)
      audio_320, sr_320 = torchaudio.load(mp3_path_320)
      audio_128, sr_128 = torchaudio.load(mp3_path_128)

      f1,f2 = None, None
      if root_dir == self.root_dir:
        f1 = self.tp.submit(self.save_file, mp3_path_320 ,os.path.join('/content/data/mp3s/320mp3/'))
        f2 = self.tp.submit(self.save_file, mp3_path_128 ,os.path.join('/content/data/mp3s/128mp3/'))

      channel = idx % 2
      channel = min(channel,audio_320.shape[0]-1)
      segment_len = 44100*8

      while audio_320.shape[-1] < segment_len:
            audio_320 = torch.cat([audio_320, audio_320[:segment_len - audio_320.shape[-1]]], dim=-1)
            audio_128 = torch.cat([audio_128, audio_128[:segment_len - audio_128.shape[-1]]], dim=-1)

      assert audio_320.shape[0] == audio_128.shape[0]
      assert audio_320.shape[1] >= segment_len
      assert audio_128.shape[1] >= segment_len

      random_start = torch.randint(0, audio_320.shape[1] - segment_len, (1,))


      audio_320 = audio_320[channel, random_start:random_start+segment_len]
      audio_128 = audio_128[channel, random_start:random_start+segment_len]

      # if f1 is not None:
      #   f1.result()
      # if f2 is not None:
      #   f2.result()

      return audio_320, audio_128
    except:
      import traceback
      print(mp3_path_320)
      print(mp3_path_128)
      print(traceback.print_exc())
      print(f'Error on index {idx}')
      return None

def collate_fn(batch):
  batch = list(filter(lambda x: (x is not None) and (len(x[0]) == 44100*8), batch))
  audio_320, audio_128 = zip(*batch)
  audio_320 = torch.stack(audio_320)
  audio_128 = torch.stack(audio_128)
  return audio_320, audio_128
# Usage
dataset = PairedAudioDataset('/content/drive/MyDrive/mp3_superresulution/dataset/fma')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=15, shuffle=True,
                                          num_workers=4, collate_fn=collate_fn)

# for i in tqdm.tqdm(range(100)):
#   d = dataset[i][0].shape
# print('after cache')
# for i in tqdm.tqdm(range(100)):
#   d = dataset[i][0].shape

# for batch in data_loader:
#   audio_320, audio_128 = batch
#   break


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
# d = dataset[28355][0].shape


In [7]:
# torchaudio.load('/content/data/mp3s/128mp3/116366.mp3')

In [8]:
# o = dataset[2012]

In [9]:
# o.shape[0]

In [10]:
from denoiser.demucs import Demucs

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
dmodel = Demucs(sample_rate=44100, hidden=32, kernel_size=16, stride=8, max_hidden=5000,depth=4).to(device)

In [13]:

dmodel.load_state_dict(torch.load('../../demucs_model.pt'))

<All keys matched successfully>

In [14]:
from denoiser.stft_loss import MultiResolutionSTFTLoss


In [15]:
stft_loss_fn = MultiResolutionSTFTLoss().to(device)

In [16]:
import tqdm

In [18]:
# prompt: create a training loop with l1 loss that trains the model to output  as close as possible to audio_320 given audio_128

import torch.optim as optim

# Define the loss function
loss_fn = torch.nn.L1Loss()

# Define the optimizer
optimizer = optim.Adam(dmodel.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
  for i, batch in tqdm.tqdm(enumerate(data_loader),total=len(data_loader)):
    if batch is None:
      continue
    audio_320, audio_128 = batch
    audio_320, audio_128 = audio_320.to(device), audio_128.to(device)

    # Forward pass
    output = dmodel(audio_128).squeeze(1)

    # Calculate the loss
    sc_loss, mag_loss  = stft_loss_fn(output, audio_320)
    loss = loss_fn(output, audio_320) + sc_loss + mag_loss

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss
    if i % 100 == 0:
      print(f"Epoch: {epoch+1}/{10}, Batch: {i+1}/{len(data_loader)}, Loss: {loss.item():.4f}")
  torch.save(dmodel.state_dict(), f'../../demucs_model_{epoch}.pt')

  0%|          | 1/2800 [00:23<18:20:46, 23.60s/it]

Epoch: 1/10, Batch: 1/2800, Loss: 0.0853


  4%|▎         | 101/2800 [04:00<1:49:12,  2.43s/it]

Epoch: 1/10, Batch: 101/2800, Loss: 0.0722


  7%|▋         | 186/2800 [06:58<1:40:17,  2.30s/it]

...

Error on index 18965

  File "/usr/local/lib/python3.10/dist-packages/torchaudio/_backend/ffmpeg.py", line 69, in _load_audio
    s.process_all_packets()


  File "/usr/local/lib/python3.10/dist-packages/torio/io/_streaming_media_decoder.py", line 901, in process_all_packets
    self._be.process_all_packets()
KeyboardInterrupt


None
Error on index 18775


KeyboardInterrupt: 

In [19]:
torch.save(dmodel.state_dict(), '../../demucs_model.pt')

In [20]:
output.shape

torch.Size([15, 352800])

In [21]:
output[0][0]

tensor(-0.0345, device='cuda:0', grad_fn=<SelectBackward0>)

In [22]:
import IPython.display as ipd

In [26]:
ipd.Audio(audio_128[2].detach().cpu().numpy(), rate=44100)

In [27]:
ipd.Audio(output[2].detach().cpu().numpy(), rate=44100)

In [25]:
for i in range(30):
  print((audio_320[i] - output[i][0]).abs().mean())
  print((audio_128[i] - audio_320[i][0]).abs().mean())
  print('#########################################')

tensor(0.1040, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3211, device='cuda:0')
#########################################
tensor(0.1176, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1117, device='cuda:0')
#########################################
tensor(0.1212, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1931, device='cuda:0')
#########################################
tensor(0.1385, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1446, device='cuda:0')
#########################################
tensor(0.0145, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0247, device='cuda:0')
#########################################
tensor(0.1427, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1718, device='cuda:0')
#########################################
tensor(0.1473, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1698, device='cuda:0')
#########################################
tensor(0.2622, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2753, device='cud

IndexError: index 15 is out of bounds for dimension 0 with size 15

dataset

In [1]:
cd /content/drive/MyDrive/mp3_superresulution/dataset/fma

/content/drive/MyDrive/mp3_superresulution/dataset/fma


In [27]:
pip install python-dotenv

In [2]:
%matplotlib inline

import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils


In [3]:
import pandas as pd

In [4]:
tracks = utils.load('data/fma_metadata/tracks.csv')
genres = utils.load('data/fma_metadata/genres.csv')
features = utils.load('data/fma_metadata/features.csv')
echonest = utils.load('data/fma_metadata/echonest.csv')

In [5]:
df = tracks['track']

In [32]:
# df.sort_values('number')

In [33]:
pip install mutagen

In [34]:
pip install pydub

In [6]:
from mutagen.mp3 import MP3


In [7]:
import pydub
import glob
import tqdm

In [8]:
cd data/mp3s/

/content/drive/MyDrive/mp3_superresulution/dataset/fma/data/mp3s


In [9]:
files = list(glob.glob('fma_large/*/*.mp3'))

In [10]:
len(files)

106574

In [11]:
rates = df.bit_rate.values

In [ ]:
for i,path in enumerate(tqdm.tqdm(files)):
  try:
    name = path.split('/')[-1]
    # if os.path.exists(f'320mp3/{name}') and os.path.exists(f'128mp3/{name}'):
    #   continue
    if os.path.exists(f'32mp3/{name}'):
      continue
    if rates[i] == 320000:
      # f = MP3(path)
      # if f.info.bitrate > 300000:
      if os.path.exists(f'320mp3/{name}') :
          song = pydub.AudioSegment.from_mp3(path)
          # song.export(f'320mp3/{name}', format="mp3", bitrate="320k")
          # song.export(f'128mp3/{name}', format="mp3", bitrate="128k")
          song.export(f'32mp3/{name}', format="mp3", bitrate="32k")

          if f.info.bitrate != 320000:
            print(f.info.bitrate)
      else:
        pass
  except:
      pass

 48%|████▊     | 51413/106574 [1:29:00<7:43:40,  1.98it/s] 

In [ ]:
 MP3(path).info.bitrate

In [ ]:
path

In [ ]:
df.head(102)

In [ ]:
print(f'128mp3/{name}')

In [ ]:
MP3('320mp3/000171.mp3').info.bitrate

In [ ]:
f.info.bitrate

In [ ]:
ls fma_large/000/000002.mp3

In [ ]:
df.groupby('bit_rate').count()['comments'].sort_values(ascending=False)

In [ ]:
# df

In [ ]:
df.groupby('bit_rate').count()['comments'].sort_values(ascending=False)

In [ ]:
!wget https://os.unil.cloud.switch.ch/fma/fma_large.zip

In [ ]:
!unzip fma_large.zip

In [ ]:
ls

In [ ]:
# df = pd.read_csv('fma_metadata/features.csv')

In [ ]:
import librosa

In [ ]:

w,sr = librosa.load('/content/Metallica_ Nothing Else Matters (Official Music Video) (320).mp3',sr=None)

In [ ]:

w_low,sr_low = librosa.load('/content/Metallica_ Nothing Else Matters (Official Music Video) (low).mp3',sr=None)

In [ ]:
import numpy as np

In [ ]:
import plotly.express as px

In [ ]:
import IPython.display as ipd

In [ ]:
import soundfile as sf
sf.write('test.wav',w-w_low,sr)

In [ ]:
px.line(x=np.arange(100000),y=(w-w_low)[:100000] ** 2)